In [1]:
import mysql.connector
import requests
from bs4 import BeautifulSoup

# establish connection
cnx = mysql.connector.connect(user='root', password='',
                              host='localhost', database='users')

# create cursor object
cursor = cnx.cursor()

# execute a query to get the data to be updated
query = "SELECT ID, NAME, LAST_UPDATED FROM touristic_sites"
cursor.execute(query)

# iterate over results
for result in cursor:
    site_id, site_name, db_last_updated = result

    # Send a GET request to Google's search engine
    query = f'{site_name} Lebanon'
    num_results = 1  # number of search results to retrieve
    url = f'https://www.google.com/search?q={query}&num={num_results}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)

    # Parse the HTML response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'lxml')

    # Locate the search results container
    search_results = soup.find_all('div', class_='g')

    # Extract the necessary data from each search result
    urls = []
    queries=[]
    for result in search_results:
        # Extract the URL of the search result
        url = result.find('a')['href']
        # Check if the URL is a real link and add it to the list of URLs
        if url.startswith('http'):
            urls.append(url)

    # Query each website and extract the last updated time
    last_updated_time = None
    for url in urls:
        # Send a GET request to the website
        response = requests.get(url)

        # Parse the HTML response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'lxml')

        # Locate the last updated time, if available
        time_tag = soup.find('time')
        if time_tag:
            last_updated_time = time_tag.text
            break

    # Update the database if a new last updated time was found
    if last_updated_time and last_updated_time != db_last_updated:
        update_query = f"UPDATE touristic_sites SET LAST_UPDATED = '{last_updated_time}' WHERE ID = {site_id}"
        cursor.execute(update_query)
        cnx.commit()
        print(f"{site_name}: Last Updated (DB): {db_last_updated}, Last Updated (Web): {last_updated_time}")
    else:
        print(f"{site_name}: Last Updated (DB): {db_last_updated}, Last Updated (Web): {last_updated_time or 'not found'}")

cursor.close()
cnx.close()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Touristic sites in lebanon' at line 1

In [ ]:
cursor.close()
cnx.close()